In [200]:
import os
import pandas as pd
import re

def masterfile():
    # Paths
    folderPath = "FOLDER"  # Update this
    masterPath = "Master datasheet.xlsx"  # Update this
    outputPath = "master/master.xlsx"  # Update this

    def generateCol(n):
        """ Generate Excel-style column names (A, B, C... AA, AB, etc.). """
        names = []
        for i in range(n):
            col_name = ''
            while i >= 0:
                col_name = chr(i % 26 + 65) + col_name
                i = i // 26 - 1
            names.append(col_name)
        return names

    # Read the master template
    master = pd.read_excel(masterPath)
    masterColnames = master.columns 
    # print(master.columns) # Preserve original column names
    n = len(master.columns)
    # print(n, masterColnames)
    Colnames = generateCol(n)
    master.columns = Colnames
    # print(master.columns)  # Use temporary column names for processing

    # Loop through all folders in the directory
    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)
        if not os.path.isdir(paths):  # Skip files, only process directories
            continue
        
        for filename in os.listdir(paths):
            filePath = os.path.join(paths, filename)

            try:
                # Read the column mapping sheet (handle missing sheets)
                try:
                    Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                except ValueError:
                    print(f"Warning: Sheet '{foldername}' not found in master template. Skipping.")
                    continue

                if not {'From', 'To'}.issubset(Col_directions.columns):
                    print(f"Error: Mapping sheet {foldername} must contain 'From' and 'To' columns.")
                    continue

                # Read the file to process
                fileRead = pd.read_excel(filePath)
                temp_data = []  # Temporary storage for processed rows
                def extract_dimensions(measurement):
                    match = re.match(r"(\d+\.?\d*)\D*(\d+\.?\d*)\D*(\d+\.?\d*)", str(measurement))
                    return match.groups() if match else (None, None, None)

                # Breaking measurement columns into M1, M2, M3
                if 'Measurement' in fileRead.columns:
                    # match = re.match(r"(\d+\.?\d*)\D*(\d+\.?\d*)\D*(\d+\.?\d*)")
                    fileRead[['m1', 'm2', 'm3']] = fileRead['Measurement'].apply(lambda x: pd.Series(extract_dimensions(x)))
                    # display(fileRead)

                    # fileRead[['m1', 'm2', 'm3']] = fileRead['Measurement'].str.split(' x ', expand=True)
                    # print(fileRead[['m1', 'm2', 'm3']])
                    # fileRead[['m1', 'm2', 'm3']] = fileRead[['m1', 'm2', 'm3']].apply(pd.to_numeric, errors='coerce')
                    # display(fileRead)
                    

               
           
                if 'Diameter' in fileRead.columns:
                    fileRead[['m1', 'm2', 'm3']] = fileRead['Diameter'].apply(lambda x: pd.Series(extract_dimensions(x)))
                    # display(fileRead)
             
                # Assign dynamic column names to the current file
                n = len(fileRead.columns)
                print(n)
                Colnames = generateCol(n)
                fileRead.columns = Colnames
                # print(fileRead.columns)
             

                # Process rows based on column mapping
                for _, row in fileRead.iterrows():
                    # if pd.isna(row['A']):  # Stop processing if empty row
                    #     break

                    new_row = {'A': filename}  # Track file name for reference

                    for _, movement_row in Col_directions.iterrows():
                        from_col = movement_row['From']
                        to_col = movement_row['To']

                        if from_col in row.index and to_col in master.columns:
                            new_row[to_col] = row[from_col]

                    temp_data.append(new_row)  # Add processed row
                    # print(temp_data)

                # Append processed data to master
                temp_df = pd.DataFrame(temp_data)
                master = pd.concat([master, temp_df], ignore_index=True)
                # display(master)

            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue
            
    # Function to calculate 'R' column
    def calculate_R(row):
        if 'P' in row and 'Q' in row and pd.notna(row['P']) and pd.notna(row['Q']):
            # print(row['Q'],filename)
            return ((100 - float(row['Q'])) / 100) * float(row['P']) if row['Q'] > 0 else ((100 + row['Q']) / 100) * float(row['P'])
        return None

    # Function to calculate 'S' column
    def calculate_S(row):
        if 'R' in row and 'C' in row and pd.notna(row['R']) and pd.notna(row['C']):
            return round(row['R'] * row['C'])  # Perform calculation
        return None

    # Apply calculations
    if 'P' in master.columns and 'Q' in master.columns:
        # print(master['R'])
        master['R'] = master.apply(calculate_R, axis=1)
    if 'R' in master.columns and 'C' in master.columns:
        master['S'] = master.apply(calculate_S, axis=1)

    # Clean column 'I'
    if 'I' in master.columns:
        master['I'] = master['I'].apply(lambda x: None if pd.isna(x) or str(x).strip() == '' else x)

    # Restore original column names
    master.columns = masterColnames

    # Save the final master file
    os.makedirs(os.path.dirname(outputPath), exist_ok=True)  # Ensure directory exists
    master.to_excel(outputPath, index=False)
    print(f"Master file updated and saved at {outputPath}.")



In [201]:
import os
import pandas as pd
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils import column_index_from_string

def hyperlink():
    folderPath = "FOLDER"
    masterPath = "Master datasheet.xlsx"
    outputPath = "master/master.xlsx"
    targetCol = column_index_from_string('X')  # Column A for data
    

    rowCount = 2
    hyperlink_pattern =  re.compile(r'HYPERLINK\("([^"]+)"')

    if os.path.exists("master\\master.xlsx"):
        # print('hello')
        target = load_workbook("master\\master.xlsx")
        targetSheet = target.active
        # print(targetSheet)
     # Add headers

    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)

        for filename in os.listdir(paths):
            try:
                filePath = os.path.join(paths, filename)
                Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                source = load_workbook(filePath)
                sourceSheet = source.active
                # print(sourceSheet)

                sourceCol = column_index_from_string(Col_directions['Hfrom'][0])
                RefCol = column_index_from_string(Col_directions['StkRf'][0])
                

                for count, row in enumerate(sourceSheet.iter_rows(min_row=2), start=2):
                    
                    cell = row[sourceCol - 1]
                    RefCell = row[RefCol-1]
                    targetCell = targetSheet.cell(row=rowCount, column=targetCol)
                    targetCell.value = cell.value
                    print(cell.value)
                    
                    if cell.hyperlink:
                        print('hello')
                        targetCell.hyperlink = cell.hyperlink
                        targetCell.style = "Hyperlink"
                    

                    if cell.data_type == 'f':
                        match = hyperlink_pattern.search(cell.value)
                        if match:
                            hyperlink = match.group(1)
                            print('hyperlink:', hyperlink)
                            targetCell.hyperlink = hyperlink
                            targetCell.style = "Hyperlink"
                    rowCount += 1

                print(f"Processed {filename} successfully")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    target.save(outputPath)
    print("Done")






In [204]:
import pandas as pd
import psycopg2
from openpyxl import load_workbook

def database_transfer():
    conn = psycopg2.connect(
        user="postgres",
        host="localhost",
        database="Master_Compilation",
        password="123",
        port=5432,
    )
    db = conn.cursor()

    # Create table if not exists
    db.execute('''
        CREATE TABLE IF NOT EXISTS diamonds (
            id SERIAL PRIMARY KEY,
            company TEXT,
            shape TEXT,
            size NUMERIC,
            colour TEXT,
            clarity TEXT,
            cut TEXT,
            polish TEXT,
            symmetry TEXT,
            flour TEXT,
            ratio NUMERIC,
            m1 NUMERIC,
            m2 NUMERIC,
            m3 NUMERIC,
            depth NUMERIC,
            tables NUMERIC,
            rap_rate NUMERIC,
            discount NUMERIC,
            ppc NUMERIC,
            total NUMERIC,
            total_in_rs NUMERIC,
            report_no TEXT,
            stockref TEXT,
            cert_link TEXT,
            video_link TEXT,
            binC TEXT,
            binS TEXT,
            comment TEXT,
            comment2 TEXT
        )
    ''')
    conn.commit()

    file = "master\master.xlsx"
    wb = load_workbook(file, data_only=True)
    ws = wb.active
    df = pd.read_excel(file, engine="openpyxl")
    columns = list(df.columns)
    columns.append('Date')
    
    # Extract video links from hyperlinks
    video_links = []
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=df.columns.get_loc("Video link")+1, max_col=df.columns.get_loc("Video link")+1):
        cell = row[0]
        if cell.hyperlink:
            video_links.append(cell.hyperlink.target)
        else:
            video_links.append(None)
    df['Video link'] = video_links

    # Convert numeric columns
    numeric_columns = ["Size", "Ratio", "M1", "M2", "M3", "Depth", "Table",
                       "Rap rate", "Discount", "PPC", "Total"]
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Insert data into the table
    for index, row in df.iterrows():
        db.execute(''' 
            INSERT INTO diamonds (company, shape, size, colour, clarity, cut, polish, symmetry,
                flour, ratio, m1, m2, m3, depth, tables, rap_rate, discount, ppc, total, total_in_rs, 
                report_no, stockref, cert_link, video_link, binC, binS, comment, comment2) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', tuple(row))
    
    conn.commit()
    db.close()
    conn.close()
    print("Successfully added")

database_transfer()


Successfully added


In [ ]:
import threading

t1= threading.Thread(target=masterfile)
t2 = threading.Thread(target=hyperlink)

t1.start()
t1.join()
t2.start()
t2.join()

In [ ]:
pd.read_excel("E:\\office\\masterFile\\FOLDER\\SRK\\srk3.xlsx")